# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# Chapitre 4 - Analyse en composantes principales

Ce calepin Jupyter accompage le Chapitre 4 des notes de cours.

Chargeons d'abord les librairies nécessaires.

In [ ]:
# Chargement des librairies
using CSV, DataFrames             # Pour charger et organiser les données
using Gadfly                      # Pour générer des graphiques
using LinearAlgebra               # Pour utiliser les fonctions d'algèbre linéaire
using Statistics                  # Libriairie contenant des fonctions statistiques de base

Afin de ne pas alourdir le calepin, un fichier de fonctions relatives à la standardisation des matrices est fourni. Le fichier *functions.jl* contient les fonctions suivantes :
- standardize ;
- standardize! .

Le fichier être inclus à l'aide de la commande `include("functions.jl")`. Pour avoir une description de ces fonctions, par exemple la fonction *roc_curve*, vous pouvez utiliser la commande `?roc_curve` dans une cellule de code vide.

In [ ]:
include("functions.jl");

# Exercice : Pourcentage de gras

Reprenons l'exercice sur le lien entre le pourcentage de gras et trois différentes variables explicatives :
- $x_1$ l'épaisseur du pli du triceps ;
- $x_2$ le tour de cuisse ;
- $x_3$ la circonférence du bras.

Nous avions vu qu'il y avait un problème de multicolinéarité lorsque les trois variables étaient utilisées. Dans ce calepin, nous allons effectuer
1. L'analyse en composantes principales des variables explicatives.
2. La rérgession sur composantes principales.

Chargeons d'abord les données.

In [ ]:
# Chargement des données
data = CSV.read("bodyfat.csv")

## 1. Analyse en composantes principales

- Standardisation des données.
- Décomposition en valeurs singulières de la matrice standardisée.

In [ ]:
# Extraction de la matrice des variables explicatives du DataFrame
X = convert(Matrix{Float64}, data[:,1:3])

# Standardisation des colonnes de la matrice des variables explicatives
Z = standardize(X)

# Dimension de la matrice des variables explicatives
n, p = size(Z)

In [ ]:
# Calcul des valeurs singulières de la matrice Z
γ = svdvals(Z)

In [ ]:
df = DataFrame(k = Int64[], Variance = Float64[])

cumvar = cumsum(γ.^2)

for j=1:p
    push!(df, [j, cumvar[j]/cumvar[end]])
end

df

In [ ]:
plot(df, x=:k, y=:Variance, Geom.point, Geom.line, Guide.xlabel("Nombre de composantes"),
    Guide.ylabel("Variance retenue (%)"))

In [ ]:
# Décomposition en valeurs singulières de la matrice Z
F = svd(Z)

# Récupération des vecteus singuliers V
V = collect(F.V)

In [ ]:
# Calcul des composantes principales
T = Z*V

## Régression sur composantes principales

- Estimation des coefficients de régression dans l'espace des composantes principales.
- Transformation des estimation à l'espace de la matrice standardisée Z 
- Régression en utilisant la matrice Z mais en utilisant l'estimation obtenue avec les composantes principales

In [ ]:
# Récupération de la variable d'intérêt
y = data[:,:Bodyfat]

# Standardisation des observations de la variable d'intérêt
standardize!(y)

# Régression dans l'espace des composantes principales
η̂ = (T'T)\T'y

In [ ]:
# Transformation des coefficient de régression à l'espace de la matrice Z

β̂ = V*η̂

# On utilise toutes les variables explicatives sans se soucier de la multicolinéarité.

In [ ]:
# Régression
ŷ = Z*β̂
e = y - ŷ

SST = sum(y.^2)
SSE = sum(e.^2)
R2 = 1 - SSE/SST


In [ ]:
df = DataFrame( ŷ = ŷ, e=e )
plot(df, x=:ŷ, y=:e, Geom.point)